# Toy Model 1: Agent-Based Simulation

In [1]:
import Agent

import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.collections import LineCollection
import matplotlib as mpl
import numpy as np
from scipy import optimize as opt
from scipy import integrate as intg
from scipy.stats import beta
from scipy.stats import cumfreq, beta
%matplotlib inline
plt.style.use('notebook.mplstyle')

## Population parameters

In [2]:
# Setting exogenous poluation parameters
def set_exog_params(B, d, f, a, b):
    global b0, b_vals, δ, u , α, β
    b0 = B
    b_vals = range(1,b0+1)
    δ = d
    u = f
    α, β = (a, b)
    


In [3]:
# Initial condition equation
def initial_condition(x):
    return (u(x) 
            - δ * u(x) * beta.cdf(x,α,β) 
            - δ * intg.quad(lambda t: u(t) * beta.pdf(t,α,β), x, 1)[0])

# Recurrence relation 
def phi(x):
    return (u(x) 
            - δ * u(x) * beta.cdf(x,α,β) 
            - δ * u(θ_prev)*(1-beta.cdf(θ_prev,α,β)) 
            - δ * intg.quad(lambda t : u(t) * beta.pdf(t,α,β), x, θ_prev)[0])

## Computing initial condition $\theta^*_1$ and $\left\{\theta^*_b\right\}^{b_0}_{b=1}$

In [4]:
def compute_θ1():
    return opt.root_scalar(initial_condition, bracket=[0,1], method='brentq').root

def compute_θb():
 return opt.root_scalar(phi, bracket=[0,1], method='brentq').root

def compute_θ_vals(b0):
    global θ_prev
    θ_vals = [compute_θ1()]
    for i in range(b0-1):
        θ_prev = θ_vals[i]
        θ_vals.append(compute_θb())
    return θ_vals

## Agent spawning

In [7]:
def homogeneous_spawn(n, b0, δ, u, α, β):
    set_exog_params(b0, δ, u, α, β)
    θ_vals = compute_θ_vals(b0)
    agents = [Agent(i, 0, beta.var(α, β), b0, δ, u, θ_vals) 
              for i in range(n)]
    return agents          
    
males = homogeneous_spawn(1000, 20, 0.9, lambda x : x , 1, 1)
females = homogeneous_spawn(1000, 20, 0.9, lambda x : x , 1, 1)

TypeError: 'module' object is not callable

## Tinder Simlation

In [ ]:
T = 1000
for t in range(t):
    for i in range(n):
        # Matching
        j = (i+t)%n

        # Swiping 
        m_swipe = males[i].swipe(females[j]) 
        f_swipe = females[j].swipe(males[i]) 

        # Updating
        males[i].update(m_swipe, f_swipe, females[j])
        females[j].update(f_swipe, m_swipe, males[i])

        # Killing
        # Generate a bernoulli rvar w.p delta
        # if =1: kill